In [1]:
// in Scala
val static = spark.read.json("/spark-data/activity-data/")
val dataSchema = static.schema


Intitializing Scala interpreter ...

Spark Web UI available at http://bae5ef2081fd:4041
SparkContext available as 'sc' (version = 3.0.0-preview2, master = local[*], app id = local-1620524395237)
SparkSession available as 'spark'


static: org.apache.spark.sql.DataFrame = [Arrival_Time: bigint, Creation_Time: bigint ... 8 more fields]
dataSchema: org.apache.spark.sql.types.StructType = StructType(StructField(Arrival_Time,LongType,true), StructField(Creation_Time,LongType,true), StructField(Device,StringType,true), StructField(Index,LongType,true), StructField(Model,StringType,true), StructField(User,StringType,true), StructField(gt,StringType,true), StructField(x,DoubleType,true), StructField(y,DoubleType,true), StructField(z,DoubleType,true))


In [2]:
// in Scala
val streaming = spark.readStream.schema(dataSchema)
  .option("maxFilesPerTrigger", 1).json("/spark-data/activity-data/")


streaming: org.apache.spark.sql.DataFrame = [Arrival_Time: bigint, Creation_Time: bigint ... 8 more fields]


In [3]:
// in Scala
val activityCounts = streaming.groupBy("gt").count()


activityCounts: org.apache.spark.sql.DataFrame = [gt: string, count: bigint]


In [4]:
spark.conf.set("spark.sql.shuffle.partitions", 5)


In [5]:
// in Scala
val activityQuery = activityCounts.writeStream.queryName("activity_counts")
  .format("memory").outputMode("complete")
  .start()


activityQuery: org.apache.spark.sql.streaming.StreamingQuery = org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@7205c6a0


In [6]:
//activityQuery.awaitTermination()


In [7]:
spark.streams.active


res2: Array[org.apache.spark.sql.streaming.StreamingQuery] = Array(org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@7205c6a0)


In [8]:
// in Scala
for( i <- 1 to 5 ) {
    spark.sql("SELECT * FROM activity_counts").show()
    Thread.sleep(1000)
}


+----------+-----+
|        gt|count|
+----------+-----+
|       sit|86162|
|     stand|79694|
|stairsdown|65554|
|      walk|92792|
|  stairsup|73165|
|      null|73136|
|      bike|75580|
+----------+-----+

+----------+------+
|        gt| count|
+----------+------+
|       sit|123085|
|     stand|113850|
|stairsdown| 93648|
|      walk|132560|
|  stairsup|104523|
|      null|104480|
|      bike|107973|
+----------+------+

+----------+------+
|        gt| count|
+----------+------+
|       sit|160006|
|     stand|148005|
|stairsdown|121733|
|      walk|172328|
|  stairsup|135887|
|      null|135826|
|      bike|140370|
+----------+------+

+----------+------+
|        gt| count|
+----------+------+
|       sit|209235|
|     stand|193550|
|stairsdown|159178|
|      walk|225351|
|  stairsup|177714|
|      null|177615|
|      bike|183559|
+----------+------+

+----------+------+
|        gt| count|
+----------+------+
|       sit|246159|
|     stand|227709|
|stairsdown|187264|
|      

In [9]:
// in Scala
import org.apache.spark.sql.functions.expr
val simpleTransform = streaming.withColumn("stairs", expr("gt like '%stairs%'"))
  .where("stairs")
  .where("gt is not null")
  .select("gt", "model", "arrival_time", "creation_time")
  .writeStream
  .queryName("simple_transform")
  .format("memory")
  .outputMode("append")
  .start()



import org.apache.spark.sql.functions.expr
simpleTransform: org.apache.spark.sql.streaming.StreamingQuery = org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@77cb72da


In [10]:
// in Scala
val deviceModelStats = streaming.cube("gt", "model").avg()
  .drop("avg(Arrival_time)")
  .drop("avg(Creation_Time)")
  .drop("avg(Index)")
  .writeStream.queryName("device_counts").format("memory").outputMode("complete")
  .start()


deviceModelStats: org.apache.spark.sql.streaming.StreamingQuery = org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@75907124


In [11]:
// in Scala
val historicalAgg = static.groupBy("gt", "model").avg()
val deviceModelStats = streaming.drop("Arrival_Time", "Creation_Time", "Index")
  .cube("gt", "model").avg()
  .join(historicalAgg, Seq("gt", "model"))
  .writeStream.queryName("device_counts").format("memory").outputMode("complete")
  .start()


java.lang.IllegalArgumentException:  Cannot start query with name device_counts as a query with that name is already active in this SparkSession

In [ ]:
// in Scala
// Subscribe to 1 topic
val ds1 = spark.readStream.format("kafka")
  .option("kafka.bootstrap.servers", "host1:port1,host2:port2")
  .option("subscribe", "topic1")
  .load()
// Subscribe to multiple topics
val ds2 = spark.readStream.format("kafka")
  .option("kafka.bootstrap.servers", "host1:port1,host2:port2")
  .option("subscribe", "topic1,topic2")
  .load()
// Subscribe to a pattern of topics
val ds3 = spark.readStream.format("kafka")
  .option("kafka.bootstrap.servers", "host1:port1,host2:port2")
  .option("subscribePattern", "topic.*")
  .load()


In [ ]:
// in Scala
ds1.selectExpr("topic", "CAST(key AS STRING)", "CAST(value AS STRING)")
  .writeStream.format("kafka")
  .option("checkpointLocation", "/to/HDFS-compatible/dir")
  .option("kafka.bootstrap.servers", "host1:port1,host2:port2")
  .start()
ds1.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")
  .writeStream.format("kafka")
  .option("kafka.bootstrap.servers", "host1:port1,host2:port2")
  .option("checkpointLocation", "/to/HDFS-compatible/dir")
  .option("topic", "topic1")
  .start()


In [ ]:
//in Scala
datasetOfString.write.foreach(new ForeachWriter[String] {
  def open(partitionId: Long, version: Long): Boolean = {
    // open a database connection
  }
  def process(record: String) = {
    // write string to connection
  }
  def close(errorOrNull: Throwable): Unit = {
    // close the connection
  }
})


In [ ]:
// in Scala
val socketDF = spark.readStream.format("socket")
  .option("host", "localhost").option("port", 9999).load()


In [ ]:
activityCounts.format("console").write()


In [ ]:
// in Scala
activityCounts.writeStream.format("memory").queryName("my_device_table")


In [ ]:
// in Scala
import org.apache.spark.sql.streaming.Trigger

activityCounts.writeStream.trigger(Trigger.ProcessingTime("100 seconds"))
  .format("console").outputMode("complete").start()


In [ ]:
// in Scala
import org.apache.spark.sql.streaming.Trigger

activityCounts.writeStream.trigger(Trigger.Once())
  .format("console").outputMode("complete").start()


In [ ]:
// in Scala
case class Flight(DEST_COUNTRY_NAME: String, ORIGIN_COUNTRY_NAME: String,
  count: BigInt)
val dataSchema = spark.read
  .parquet("/data/flight-data/parquet/2010-summary.parquet/")
  .schema
val flightsDF = spark.readStream.schema(dataSchema)
  .parquet("/data/flight-data/parquet/2010-summary.parquet/")
val flights = flightsDF.as[Flight]
def originIsDestination(flight_row: Flight): Boolean = {
  return flight_row.ORIGIN_COUNTRY_NAME == flight_row.DEST_COUNTRY_NAME
}
flights.filter(flight_row => originIsDestination(flight_row))
  .groupByKey(x => x.DEST_COUNTRY_NAME).count()
  .writeStream.queryName("device_counts").format("memory").outputMode("complete")
  .start()